# Project 3

In [15]:
import pandas as pd
import numpy as np
import csv
from datetime import datetime
import psycopg2 as pg
import pandas.io.sql as pd_sql

### Load data

In [16]:
connection_args = {
    'host': '13.52.182.91', #AWS's IP here
    'user': 'ubuntu',
    'dbname': 'hurricane',
    'port': 5432
}

connection = pg.connect(**connection_args)
query = "Select * from hurricane;" # if multi-line (carriage return), use triple quote '''

df = pd_sql.read_sql(query, connection)

## if all else fails...
# headers = ['Year_Month_Day', 'Hour_Minute', 'Status'
#            , 'Classification', 'Latitude', 'Longitude', 'Max_sustained_wind_knots', 'Minimum_pressure_millibars'
#            , '34ktNEMaxRadii_nautmiles', '34ktSEMaxRadii_nautmiles', '34ktSWMaxRadii_nautmiles', '34ktNWMaxRadii_nautmiles'
#            , '50ktNEMaxRadii_nautmiles', '50ktSEMaxRadii_nautmiles', '50ktSWMaxRadii_nautmiles', '50ktNWMaxRadii_nautmiles'
#            , '64ktNEMaxRadii_nautmiles', '64ktSEMaxRadii_nautmiles', '64ktSWMaxRadii_nautmiles', '64ktNWMaxRadii_nautmiles'
#           ]
# df = pd.read_table('C:/Users/tyler/Downloads/hurdat2-1851-2018-120319.txt', names=headers, delimiter=',')

,Year_Month_Day,Hour_Minute,Status,Classification,Latitude,Longitude,Max_sustained_wind_knots,Minimum_pressure_millibars,34ktNEMaxRadii_nautmiles,34ktSEMaxRadii_nautmiles,34ktSWMaxRadii_nautmiles,34ktNWMaxRadii_nautmiles,50ktNEMaxRadii_nautmiles,50ktSEMaxRadii_nautmiles,50ktSWMaxRadii_nautmiles,50ktNWMaxRadii_nautmiles,64ktNEMaxRadii_nautmiles,64ktSEMaxRadii_nautmiles,64ktSWMaxRadii_nautmiles,64ktNWMaxRadii_nautmiles
0,AL011851,UNNAMED,14,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18510625,0000,,HU,28.0N,94.8W,80.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,18510625,0600,,HU,28.0N,95.4W,80.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3,18510625,1200,,HU,28.0N,96.0W,80.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
4,18510625,1800,,HU,28.1N,96.5W,80.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53214,20181103,1200,,EX,57.9N,19.6W,55.0,960.0,780.0,600.0,660.0,480.0,0.0,240.0,0.0,0.0,0.0,0.0,0.0,0.0
53215,20181103,1800,,EX,58.9N,17.1W,50.0,964.0,480.0,600.0,660.0,420.0,0.0,240.0,0.0,0.0,0.0,0.0,0.0,0.0
53216,20181104,0000,,EX,59.8N,14.5W,45.0,968.0,360.0,480.0,480.0,360.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
53217,20181104,0600,,EX,60.8N,12.1W,40.0,973.0,270.0,330.0,360.0,270.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Data Cleaning

### Isolating Title Row data and putting it into its own column

In [17]:
basin_cyclonenumber_year_list = [np.nan]*df.shape[0]
name_list = [np.nan]*df.shape[0]
rowcount_list = [np.nan]*df.shape[0]

def hurricane_info_finder(row):
    '''
    This data set has 2 row-types: title rows and data rows. Details here:
    https://www.nhc.noaa.gov/data/hurdat/hurdat2-format-atlantic.pdf
    
    This function isolates title rows, putting all the title-row data into a column of their own.
    '''
    
    index = row.name
    
    # these variables are the data represented in title rows
    basin_cyclonenumber_year = row['Year_Month_Day']
    name = row['Hour_Minute']
    rowcount = row['Status']
    
    try:
        if np.isnan(row['Max_sustained_wind_knots']):    # Max_sustained_wind_knots is always null in title rows
            basin_cyclonenumber_year_list[index] = basin_cyclonenumber_year
            name_list[index] = name
            rowcount_list[index] = rowcount
    except:
        pass
        
df.apply(hurricane_info_finder, axis = 1)

0        None
1        None
2        None
3        None
4        None
         ... 
53214    None
53215    None
53216    None
53217    None
53218    None
Length: 53219, dtype: object

### Setup data to be merged
Put the new columns into a dictionary, then convert the dictionary into a DF, then forward fill values

In [18]:
new_column_dict = {
    'Basin_CycloneNumber_Year': basin_cyclonenumber_year_list, 
    'Name': name_list, 
    'RowCount': rowcount_list
}

df2 = pd.DataFrame.from_dict(new_column_dict)
df2 = df2.ffill() # forward fill title rows to its perspective data rows

### Merge it
Using what I'm calling a 'blind merge' becase every row will line up between both data frames

In [19]:
def blind_merge(df_left, df_right):
    '''
    Merge df_right onto df_left without considering index
    or any other merge conditions.
    Both dfs need to be mirrors.
    '''
    for column in df_right:
        df_left[column] = df_right[column]
        
blind_merge(df, df2)

### Drop "title rows"
As commented before, Latitude (and many other vlaues) is only null on Title Rows

In [20]:
df.dropna(subset=['Max_sustained_wind_knots'], inplace=True)
df.reset_index(inplace=True)

___

In [21]:
df

,index,Year_Month_Day,Hour_Minute,Status,Classification,Latitude,Longitude,Max_sustained_wind_knots,Minimum_pressure_millibars,34ktNEMaxRadii_nautmiles,...,50ktSEMaxRadii_nautmiles,50ktSWMaxRadii_nautmiles,50ktNWMaxRadii_nautmiles,64ktNEMaxRadii_nautmiles,64ktSEMaxRadii_nautmiles,64ktSWMaxRadii_nautmiles,64ktNWMaxRadii_nautmiles,Basin_CycloneNumber_Year,Name,RowCount
0,1,18510625,0000,,HU,28.0N,94.8W,80.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,AL011851,UNNAMED,14
1,2,18510625,0600,,HU,28.0N,95.4W,80.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,AL011851,UNNAMED,14
2,3,18510625,1200,,HU,28.0N,96.0W,80.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,AL011851,UNNAMED,14
3,4,18510625,1800,,HU,28.1N,96.5W,80.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,AL011851,UNNAMED,14
4,5,18510625,2100,L,HU,28.2N,96.8W,80.0,-999.0,-999.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,AL011851,UNNAMED,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51341,53214,20181103,1200,,EX,57.9N,19.6W,55.0,960.0,780.0,...,240.0,0.0,0.0,0.0,0.0,0.0,0.0,AL162018,OSCAR,36
51342,53215,20181103,1800,,EX,58.9N,17.1W,50.0,964.0,480.0,...,240.0,0.0,0.0,0.0,0.0,0.0,0.0,AL162018,OSCAR,36
51343,53216,20181104,0000,,EX,59.8N,14.5W,45.0,968.0,360.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AL162018,OSCAR,36
51344,53217,20181104,0600,,EX,60.8N,12.1W,40.0,973.0,270.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AL162018,OSCAR,36


# Feature Generation
1. **Date features**  
2. **Adding detail to Status and Classification**  
3. **Use get_dummies() and maybe to fill forward**

#### Cleaning up current features

In [22]:
df['year'] = pd.to_numeric(df['Year_Month_Day'].str[:4])
df['month'] = pd.to_numeric(df['Year_Month_Day'].str[4:6])
df['day'] = pd.to_numeric(df['Year_Month_Day'].str[6:])
df['hour'] = pd.to_numeric(df['Hour_Minute'].str[:3])
df['minute'] = pd.to_numeric(df['Hour_Minute'].str[3:5])

# status and classification, convert to real names
status_dict = {
    'C' : 'Closest_no_landfall',
    'G' : 'Genesis',
    'I' : 'Pressure_wind_peak',
    'L' : 'Landfall',
    'P' : 'Minimum_pressure',
    'R' : 'Rapid_changes',
    'S' : 'Change_of_status',
    'T' : 'Additional_position_detail',
    'W' : 'Max_sustained_wind_speed'
}
classification_dict = {
    'TD' : 'T_Tropical_depression',
    'TS' : 'T_Tropical_storm',
    'HU' : 'T_Hurricane',
    'EX' : 'T_Extratropical_cyclone',
    'SD' : 'S_Subtropical_depression',
    'SS' : 'S_Subtropical_storm',
    'LO' : 'Low_pressure_system',
    'WV' : 'Tropical_wave',
    'DB' : 'Disturbance'
}

# strip some spaces off of Status and Classification, then map in the dictionaries
df['Status'] = df['Status'].str.strip()
df['Status'] = df.replace({"Status": status_dict})['Status']

df['Classification'] = df['Classification'].str.strip()
df['Classification'] = df.replace({"Classification": classification_dict})['Classification']

# parsing data into multiple columns
df['Lat_number'] = pd.to_numeric(df['Latitude'].str[:-1])
df['Long_number'] = pd.to_numeric(df['Longitude'].str[:-1])
df['Basin'] = df['Basin_CycloneNumber_Year'].str[:2] # this is a string
df['CycloneNumber'] = pd.to_numeric(df['Basin_CycloneNumber_Year'].str[2:4])

# get_dummies for classification and status
blind_merge(df, pd.get_dummies(df['Classification']))


#### Adding Bearing and Speed (km/hour)

In [24]:
from pygeodesy import equirectangular
from pygeodesy import bearing
# https://mrjean1.github.io/PyGeodesy/docs/frames.html

# This it pretty slow.. there's definitely a better way
# Could also make this a function
# Next iteration

speed_km_per_hour = []
bearing_list = []

for i in range(0, df.shape[0]):    
    if df.iloc[i]['Basin_CycloneNumber_Year'] == df.iloc[i-1]['Basin_CycloneNumber_Year']: # compare key to previous rec
        
        # calculate distance of movement of hurricane
        lat1 = df.iloc[i]['Lat_number']
        lon1 = df.iloc[i]['Long_number']
        lat2 = df.iloc[i-1]['Lat_number']
        lon2 = df.iloc[i-1]['Long_number']
        distance = equirectangular(lat1, lon1, lat2, lon2, limit=80)  # this returns meters
        km = distance/1000 #Kilometers
        
        # number of hours between current row and previous
        # to better calculate the speed at which the hurricane is moving
        curr_period = df.iloc[i]['Year_Month_Day'].strip() + df.iloc[i]['Hour_Minute'].strip()
        curr_period = datetime.strptime(curr_period, '%Y%m%d%H%M')
        prev_period = df.iloc[i-1]['Year_Month_Day'].strip() + df.iloc[i-1]['Hour_Minute'].strip()
        prev_period = datetime.strptime(prev_period, '%Y%m%d%H%M')
        hours = abs(curr_period - prev_period).total_seconds() / 3600.0
        
        speed_km_per_hour.append(km/hours)
        
        #angle of approach
        angle = bearing(lat1, lon1, lat2, lon2)
        bearing_list.append(angle)
        
    else:
        speed_km_per_hour.append(np.nan)
        bearing_list.append(np.nan)


print(pd.DataFrame(bearing_list).shape[0])
print(df.shape[0])

51346
51346


In [25]:
df['speed_km_per_hour'] = speed_km_per_hour
df['bearing'] = bearing_list

#### Add Hurrican Season

In [26]:
hurricane_season_dict = {
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 1,
    7: 1,
    8: 1,
    9: 1,
    10: 1,
    11: 1,
    12: 0
}

df['hurricane_season'] = df['month']
df['hurricane_season'] = df.hurricane_season.map(hurricane_season_dict)

Consider Adding These Features:  
1. distance from shore, at current angle
5. is it in the gulf, or eastern seaboard

#### Add Gulf Coast / Atlantic / Eastern Seaboard

In [27]:
# if 80W-97W and below 30 it's gulf coast
# if 80W-73W and above 26 it's Eastern Seaboard
# else Atlantic -- this part will need to be checked

## Backfill Target Values -- where Landfall occurs

In [28]:
def backfill_where_id(data, number_spaces, identifier_df):
    for row in range(1, data.shape[0]):
        for i in range(1, number_space):
            if data[row] == 'Landfall' and identifier_df[row] == identifier_df[row-i]:
                data[row-i] = 'Landfall'
            data[row] == '' # Get rid of the record where it actually occurs. We want to predict the anticipation!
            
new_col = backfill_where_id(df['Status'], 2, df['Basin_CycloneNumber_Year']) # edge case: what if the cyclone over New Years

blind_merge(df, pd.get_dummies(df['Status']))

C:\Users\tyler\Miniconda3\envs\metis\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\tyler\Miniconda3\envs\metis\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\tyler\Miniconda3\envs\metis\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [29]:
df.head(20)

,index,Year_Month_Day,Hour_Minute,Status,Classification,Latitude,Longitude,Max_sustained_wind_knots,Minimum_pressure_millibars,34ktNEMaxRadii_nautmiles,...,,Additional_position_detail,Change_of_status,Closest_no_landfall,Genesis,Landfall,Max_sustained_wind_speed,Minimum_pressure,Pressure_wind_peak,Rapid_changes
0,1,18510625,0000,,T_Hurricane,28.0N,94.8W,80.0,-999.0,-999.0,...,1,0,0,0,0,0,0,0,0,0
1,2,18510625,0600,,T_Hurricane,28.0N,95.4W,80.0,-999.0,-999.0,...,1,0,0,0,0,0,0,0,0,0
2,3,18510625,1200,Landfall,T_Hurricane,28.0N,96.0W,80.0,-999.0,-999.0,...,0,0,0,0,0,1,0,0,0,0
3,4,18510625,1800,Landfall,T_Hurricane,28.1N,96.5W,80.0,-999.0,-999.0,...,0,0,0,0,0,1,0,0,0,0
4,5,18510625,2100,Landfall,T_Hurricane,28.2N,96.8W,80.0,-999.0,-999.0,...,0,0,0,0,0,1,0,0,0,0
5,6,18510626,0000,,T_Hurricane,28.2N,97.0W,70.0,-999.0,-999.0,...,1,0,0,0,0,0,0,0,0,0
6,7,18510626,0600,,T_Tropical_storm,28.3N,97.6W,60.0,-999.0,-999.0,...,1,0,0,0,0,0,0,0,0,0
7,8,18510626,1200,,T_Tropical_storm,28.4N,98.3W,60.0,-999.0,-999.0,...,1,0,0,0,0,0,0,0,0,0
8,9,18510626,1800,,T_Tropical_storm,28.6N,98.9W,50.0,-999.0,-999.0,...,1,0,0,0,0,0,0,0,0,0
9,10,18510627,0000,,T_Tropical_storm,29.0N,99.4W,50.0,-999.0,-999.0,...,1,0,0,0,0,0,0,0,0,0


### Drop the columns that are no longer, or will not become, useful

In [30]:
# the '' column was the inverse of the Landfall column! Not sure how this happened.
df.drop(labels=['','Basin_CycloneNumber_Year', 'index', 'Year_Month_Day', 'Hour_Minute', 'Status', 'Classification'], axis=1, inplace=True)

df.drop(labels=['34ktNEMaxRadii_nautmiles', '34ktSEMaxRadii_nautmiles', '34ktSWMaxRadii_nautmiles', '34ktNWMaxRadii_nautmiles'
    , '50ktNEMaxRadii_nautmiles', '50ktSEMaxRadii_nautmiles', '50ktSWMaxRadii_nautmiles', '50ktNWMaxRadii_nautmiles'
    , '64ktNEMaxRadii_nautmiles', '64ktSEMaxRadii_nautmiles', '64ktSWMaxRadii_nautmiles', '64ktNWMaxRadii_nautmiles']
    , axis=1
    , inplace=True
)


In [31]:
# putting Landfall, the Target, at the front of the df
df = df[ ['Landfall'] + [ col for col in df.columns if col != 'Landfall' ] ]

## Export!

In [36]:
df.dtypes

Landfall                        uint8
Latitude                       object
Longitude                      object
Max_sustained_wind_knots      float64
Minimum_pressure_millibars    float64
Name                           object
RowCount                       object
year                            int64
month                           int64
day                             int64
hour                            int64
minute                          int64
Lat_number                    float64
Long_number                   float64
Basin                          object
CycloneNumber                   int64
Disturbance                     uint8
ET                              uint8
Low_pressure_system             uint8
S_Subtropical_depression        uint8
S_Subtropical_storm             uint8
T_Extratropical_cyclone         uint8
T_Hurricane                     uint8
T_Tropical_depression           uint8
T_Tropical_storm                uint8
Tropical_wave                   uint8
speed_km_per

In [32]:
df.to_pickle('C:\\Users\\tyler\\Documents\\GitHub\\sf20_ds17\\students_submissions\\ty-adams\\huricane_data')

In [38]:
from sqlalchemy import create_engine

engine = create_engine('postgresql://ubuntu@13.52.182.91:5432/hurricane')
df.to_sql('hurricane_feature_dev', engine, if_exists='replace', index=False)